In [1]:
! pip install biopython --quiet


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


In [5]:
# Core Python
import os
import re
import glob
import numpy as np
import pandas as pd
import ray

# Biopython for parsing FASTA and computing biochemical features
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis

# Optional: Visualization
import matplotlib.pyplot as plt
import seaborn as sns

2025-07-01 19:06:35,573	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [7]:
uniprot_path = "/Users/davidm/182-final-proj/data/uniref50.fasta"

from Bio import SeqIO

# Open large FASTA file as a streaming iterator
with open(uniprot_path) as handle:
    for record in SeqIO.parse(handle, "fasta"):
        # record.id -> sequence ID
        # record.seq -> actual sequence
        # Perform your analysis here
        print(record.id, len(record.seq))
        print(record)
        break



UniRef50_UPI002E2621C6 49499
ID: UniRef50_UPI002E2621C6
Name: UniRef50_UPI002E2621C6
Description: UniRef50_UPI002E2621C6 uncharacterized protein LOC134193701 n=1 Tax=Corticium candelabrum TaxID=121492 RepID=UPI002E2621C6
Number of features: 0
Seq('MGRIRVWVGTSIPNPVNAHQLVYLKGMAKTKKLILLLFVAAQPNFKEWSLDVDA...VKY')


In [13]:
from pathlib import Path

def split_fasta(fasta_path, n_chunks, output_dir="fasta_chunks"):
    Path(output_dir).mkdir(exist_ok=True)
    records = list(SeqIO.parse(fasta_path, "fasta"))
    chunk_size = len(records) // n_chunks + 1
    
    for i in range(n_chunks):
        chunk = records[i*chunk_size:(i+1)*chunk_size]
        if chunk:
            chunk_path = Path(output_dir) / f"chunk_{i}.fasta"
            SeqIO.write(chunk, chunk_path, "fasta")
    return list(Path(output_dir).glob("chunk_*.fasta"))


In [ ]:
# split_fasta(uniprot_path, 512, "uniprot_fasta_chunks")

In [3]:
from Bio.SeqUtils import gc_fraction
import ray
from Bio import SeqIO

@ray.remote
def analyze_sequence(record):
    return {
        "id": record.id,
        "length": len(record.seq),
        "gc": gc_fraction(record.seq),
    }


2025-07-01 19:04:55,106	INFO util.py:154 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [3]:
from itertools import islice

def chunked_iterator(iterator, size):
    """Yield chunks of given size from iterator."""
    while True:
        chunk = list(islice(iterator, size))
        if not chunk:
            break
        yield chunk


In [9]:
ray.init(num_cpus=6)  # use only 6 cores

2025-07-01 19:07:34,525	ERROR services.py:1340 -- Failed to start the dashboard , return code 1
2025-07-01 19:07:34,527	ERROR services.py:1365 -- Error should be written to 'dashboard.log' or 'dashboard.err'. We are printing the last 20 lines for you. See 'https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure' to find where the log file is.
2025-07-01 19:07:34,533	ERROR services.py:1409 -- 
The last 20 lines of /tmp/ray/session_2025-07-01_19-07-32_254798_70303/logs/dashboard.log (it contains the error message from the dashboard): 
    from ray.util.state.common import (
  File "/Users/davidm/opt/anaconda3/lib/python3.9/site-packages/ray/util/state/__init__.py", line 1, in <module>
    from ray.util.state.api import (
  File "/Users/davidm/opt/anaconda3/lib/python3.9/site-packages/ray/util/state/api.py", line 18, in <module>
    from ray.util.state.common import (
  File "/Users/davidm/opt/anaconda3/lib/python3.9/site-packages/ray

Python version:,3.9.13
Ray version:,2.30.0


(raylet) The node with node id: f1072cb88329fa478367faa450c0f97ace6c6f98f79e14297fc84a56 and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


(raylet) [2025-07-01 19:07:35,311 E 70414 21249175] (raylet) agent_manager.cc:83: The raylet exited immediately because one Ray agent failed, agent_name = dashboard_agent/470211272.
(raylet) The raylet fate shares with the agent. This can happen because
(raylet) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/{dashboard_agent|runtime_env_agent}.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure.
(raylet) - The agent is killed by the OS (e.g., out of memory).


In [10]:
# Parallelize over chunks instead of individual records
@ray.remote
def analyze_chunk(chunk):
    return [gc_fraction(record.seq) for record in chunk]

with open(uniprot_path) as f:
    chunks = list(chunked_iterator(SeqIO.parse(f, "fasta"), 1000))

futures = [analyze_chunk.remote(chunk) for chunk in chunks]
gc_lists = ray.get(futures)

flat_gc = [gc for sublist in gc_lists for gc in sublist]
print(f"Avg GC: {sum(flat_gc)/len(flat_gc):.2%}")


KeyboardInterrupt: 